In [211]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import re
from datetime import datetime
import json

In [212]:
URL_PREFIXO = 'https://www.mylaptime.com/laptime/clientes/9T00V814PC29X0160812G781W/results/'
URL_PRINCIPAL = 'r3.html?evt=11611&epg=8993'

In [213]:
dados_bateria = {}

In [214]:
def get_urls(url):
   
        tomada = None
        corrida = None

        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        driver = webdriver.Chrome(options=options)
        
        driver.get(url)
        time.sleep(1)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        links = soup.find_all('a', href=True)
        
        for link in links:
            if "TOMADA DE TEMPO" in link.get_text():
                 tomada = link['href']
                
        for link in links:
            if "CORRIDA" in link.get_text():
                corrida = link['href']


        div_titulo_bateria = soup.find('div', class_='StackedContainers')
        titulo_bateria = div_titulo_bateria.find('h3').get_text() if div_titulo_bateria and div_titulo_bateria.find('h3') else None

        padrao_data = re.compile(r'(\d{2})/(\d{2})/(\d{4})')
        padrao_tempo = re.compile(r'(\d{2}:\d{2})')

        # Encontrar a data e hora na string
        data = padrao_data.search(titulo_bateria)
        tempo = padrao_tempo.search(titulo_bateria)

        if data and tempo:
            dia, mes, ano = map(int, data.groups())
            tempo_str = tempo.group(1).replace(':', '')
            data_tempo_str = f"{ano}{mes:02d}{dia:02d}{tempo_str}"
            
        return tomada, corrida, data_tempo_str

In [215]:
def get_tabelas(url_tomada, url_corrida):

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)

    driver.get(URL_PREFIXO + url_tomada)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tabela_1 = extrair_dados_tabela(soup.find('table', class_='evenodd'))
    
    driver.get(URL_PREFIXO + url_tomada.replace('r4.html', 'vlts.asp'))
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tabela_2 = extrair_dados_tabela(soup.find('table', class_='evenodd'))
    
    driver.get(URL_PREFIXO + url_corrida)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tabela_3 = extrair_dados_tabela(soup.find('table', class_='evenodd'))
    
    driver.get(URL_PREFIXO + url_corrida.replace('r4.html', 'vlts.asp'))
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tabela_4 = extrair_dados_tabela(soup.find('table', class_='evenodd'))

    return {'tomada_tempo': tabela_1, 'tomada_tempo_mapa_de_voltas': tabela_2, 'corrida': tabela_3, 'corrida_mapa_de_voltas': tabela_4}

def extrair_dados_tabela(tabela):
    dados = []

    for linha in tabela.find_all('tr'):
        dados_linha = []
        for celula in linha.find_all(['th', 'td']):
            dados_linha.append(celula.get_text().strip())
        dados.append(dados_linha)
    
    return  pd.DataFrame(dados[1:], columns=dados[0])

In [216]:
def converter_volta(tempo):
    if tempo.find('VOLTA') != -1:
        if tempo == '-1 VOLTA(S)':
            return '-1 VOLTA'
        else:
            return tempo[:-3] + 'S'
    else:
        return tempo

def converter_tempo_para_ms(tempo):
    if tempo is None:
        return None
    elif tempo.count(':') == 1:
        partes = tempo.split(':')
        try:
            minutos = int(partes[0])
            segundos = int(float(partes[1]) * 1000)
        except ValueError:
            return None
        return minutos * 60 * 1000 + segundos
    elif tempo.count(':') == 2:
        partes = tempo.split(':')
        try:
            horas = int(partes[0])
            minutos = int(partes[1])
            segundos = int(float(partes[2]) * 1000)
        except ValueError:
            return None
        return horas * 60 * 60 * 1000 + minutos * 60 * 1000 + segundos
    return tempo
    

def converter_ms_para_tempo(ms):
    if ms is None:
        return None
    
    try:
        total_seconds = ms / 1000.0
        horas = int(total_seconds // 3600)
        rem = total_seconds % 3600
        
        minutos = int(rem // 60)
        segundos = rem % 60
        
        if horas > 0:
            return "{:02d}:{:02d}:{:06.3f}".format(horas, minutos, segundos)
        else:
            return "{:02d}:{:06.3f}".format(minutos, segundos)
    
    except Exception:
        return None

def converter_vm(vel):
    return float(vel.replace(',', '.'))

In [217]:
def converter_tabela_corrida(df):
    df['VM'] = df['VM'].apply(converter_vm)
    df['DL'] = df['DL'].apply(converter_volta)
    df['DA'] = df['DA'].apply(converter_volta)
    df['TMV_ms'] = df['TMV'].apply(converter_tempo_para_ms)
    df['TT_ms'] = df['TT'].apply(converter_tempo_para_ms)
    df['DL_ms'] = df['DL'].apply(converter_tempo_para_ms)
    df['DA_ms'] = df['DA'].apply(converter_tempo_para_ms)

In [218]:
url_tomada, url_corrida, bateria = get_urls(URL_PREFIXO + URL_PRINCIPAL)

In [219]:
tabelas = get_tabelas(url_tomada, url_corrida)

In [220]:
tabelas['corrida'].columns = ['POS', '#', 'CAT', 'NOME', 'MV', 'TMV', 'VLTS', 'TT', 'DL', 'DA', 'VM']
tabelas['tomada_tempo'].columns = ['POS', '#', 'CAT', 'NOME', 'MV', 'TMV', 'VLTS', 'TT', 'DL', 'DA', 'VM']

In [221]:
converter_tabela_corrida(tabelas['corrida'])
converter_tabela_corrida(tabelas['tomada_tempo'])

In [222]:
def find_min_time(df):
    time_columns = df.columns[1:]
    min_time = df[time_columns].stack().dropna().min()
    
    return min_time

In [223]:
MELHOR_VOLTA = find_min_time(tabelas['corrida_mapa_de_voltas'])

In [224]:
tabelas['corrida']['MVC'] = tabelas['corrida']['TMV'] == MELHOR_VOLTA

In [225]:
colunas = ['NOME', '#', 'POS', 'MV', 'TMV', 'VLTS', 'TT', 'DL', 'DA', 'VM', 'TMV_ms', 'TT_ms', 'DL_ms', 'DA_ms', 'MVC']
json_corrida = tabelas['corrida'].to_dict(orient='records')
json_tomada_tempo = tabelas['tomada_tempo'].to_dict(orient='records')

In [226]:
for piloto in json_corrida:
    number = piloto['#']
    if number in tabelas['corrida_mapa_de_voltas'].columns:
        piloto['MAPA_VOLTAS'] = tabelas['corrida_mapa_de_voltas'][number].tolist()
        piloto['MAPA_VOLTAS_ms'] = tabelas['corrida_mapa_de_voltas'][number].map(converter_tempo_para_ms).tolist()
        piloto['MAPA_VOLTAS_ACC_ms'] = np.cumsum(piloto['MAPA_VOLTAS_ms']).tolist()
        piloto['MAPA_VOLTAS_ms'] = [x if not np.isnan(x) else None for x in piloto['MAPA_VOLTAS_ms']]
        piloto['MAPA_VOLTAS_ACC_ms'] = [x if not np.isnan(x) else None for x in piloto['MAPA_VOLTAS_ACC_ms']]
        piloto['MAPA_VOLTAS_ACC'] = list(map(converter_ms_para_tempo,piloto['MAPA_VOLTAS_ACC_ms']))

for piloto in json_tomada_tempo:
    number = piloto['#']
    if number in tabelas['tomada_tempo_mapa_de_voltas'].columns:
        piloto['MAPA_VOLTAS'] = tabelas['tomada_tempo_mapa_de_voltas'][number].tolist()
        piloto['MAPA_VOLTAS_ms'] = tabelas['tomada_tempo_mapa_de_voltas'][number].map(converter_tempo_para_ms).tolist()
        piloto['MAPA_VOLTAS_ms'] = [x if not np.isnan(x) else None for x in piloto['MAPA_VOLTAS_ms']]

In [227]:
json_data = {
    "url": URL_PREFIXO + URL_PRINCIPAL,
    "corrida": json_corrida,
    "tomada_de_tempo": json_tomada_tempo
}

file_name = f"../data/baterias/{bateria}.json"

with open(file_name, 'w') as file:
    json.dump(json_data, file, indent=4)

print(f"O arquivo {file_name} foi salvo com sucesso.")

O arquivo ../data/baterias/202403201815.json foi salvo com sucesso.
